In [1]:
import database
from database import ModelReader
from config import config
from pyntcloud import PyntCloud

import numpy as np
import vispy
import vispy.gloo as gloo
from vispy import app
from vispy.util.transforms import perspective, translate, rotate

# load the vispy bindings for the IPython notebook which enables webGL
%load_ext vispy.ipython
%reload_ext vispy.ipython

/usr/local/lib/python3.5/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Vispy IPython module has loaded successfullyVispy IPython module has loaded successfully

In [2]:
mr = ModelReader()

In [3]:
batch = mr.getGraspBatch(32)
one_grasp = batch[1]
params = config(section='data')

scale, model_path = mr.getModelInfo(one_grasp["scaled_model_id"])
model_file = open(params['model_dir'] + model_path)

ValueError: too many values to unpack (expected 2)

In [4]:
m = PyntCloud.from_file(params['model_dir'] + model_path)
m.plot(mesh=True, backend="matplotlib")
m = m.get_sample("points_random", n = len(m.points), as_PyntCloud=True)
m.plot(backend="matplotlib")

    
contacts = one_grasp['grasp_contacts']
contacts = [contacts[x:x+3] for x in range(0,len(contacts),3)]

verts, faces = database.read_off(model_file)
database.show_model_points(verts)

database.show_two_plots(getXYZ(verts,scale*rescale), getXYZ(contacts))


NameError: name 'model_path' is not defined

In [24]:
VERT_SHADER = """
uniform mat4 u_model;
uniform mat4 u_view;
uniform mat4 u_projection;
attribute vec3 a_position;
attribute float a_id;
varying float v_id;
void main (void) {
    v_id = a_id;
    gl_Position = u_projection * u_view * u_model * vec4(a_position,1.0);
}
"""

FRAG_SHADER = """
varying float v_id;
void main()
{
    float f = fract(v_id);
    // The second useless test is needed on OSX 10.8 (fuck)
    if( (f > 0.0001) && (f < .9999) )
        discard;
    else
        gl_FragColor = vec4(1,0,0,1);
}
"""

In [31]:

import sys
import numpy as np
from vispy import scene
from vispy import app, gloo, visuals, io, geometry
from vispy.geometry import create_cube
from vispy.visuals.transforms import (STTransform, MatrixTransform,
                                      ChainTransform)

class Canvas(app.Canvas):
    def __init__(self):
        app.Canvas.__init__(self, size=(512, 512), title='Colored cube',
                            keys='interactive')

        # Build cube data
        V, I, _ = create_cube()
        vertices = VertexBuffer(V)
        self.indices = IndexBuffer(I)

        # Build program
        self.program = Program(vertex, fragment)
        self.program.bind(vertices)

        # Build view, model, projection & normal
        view = translate((0, 0, -5))
        model = np.eye(4, dtype=np.float32)
        self.program['model'] = model
        self.program['view'] = view
        self.phi, self.theta = 0, 0
        gloo.set_state(clear_color=(0.30, 0.30, 0.35, 1.00), depth_test=True)

        self.activate_zoom()

        self.timer = app.Timer('auto', self.on_timer, start=True)

        self.show()

    def on_draw(self, event):
        gloo.clear(color=True, depth=True)
        self.program.draw('triangles', self.indices)

    def on_resize(self, event):
        self.activate_zoom()

    def activate_zoom(self):
        gloo.set_viewport(0, 0, *self.physical_size)
        projection = perspective(45.0, self.size[0] / float(self.size[1]),
                                 2.0, 10.0)
        self.program['projection'] = projection

    def on_timer(self, event):
        self.theta += .5
        self.phi += .5
        self.program['model'] = np.dot(rotate(self.theta, (0, 0, 1)),
                                       rotate(self.phi, (0, 1, 0)))
        self.update()

app.run()
        
# meshdata = vispy.geometry.MeshData(vertices=np.asarray(verts), faces=faces)


In [32]:
c = Canvas()

NameError: name 'VertexBuffer' is not defined